In [1]:
import folium
from folium import plugins
from folium.features import DivIcon
import json
import geopandas as gpd
import pandas as pd

In [2]:
df_coor = pd.read_csv('municipios_geo_coordenadas.csv', engine='c')
df_pop = pd.read_csv('populacao_3.csv', engine='c')

In [3]:
df_pop.id = df_pop.id.astype('str')

In [4]:
df_coor

,id,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,1050,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,4001,34,America/Sao_Paulo
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,9201,62,America/Sao_Paulo
3,3100203,Abaeté,-19.15510,-45.4444,0,31,4003,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo
...,...,...,...,...,...,...,...,...,...
5565,2933604,Xique-Xique,-10.82300,-42.7245,0,29,3971,74,America/Sao_Paulo
5566,2517407,Zabelê,-8.07901,-37.1057,0,25,542,83,America/Sao_Paulo
5567,3557154,Zacarias,-21.05060,-50.0552,0,35,2973,18,America/Sao_Paulo
5568,2114007,Zé Doca,-3.27014,-45.6553,0,21,1287,98,America/Sao_Paulo


In [5]:
dicionario={
    'RO':11,'AC':12,'AM':13,'RR':14,'PA':15,'AP':16,
    'TO':17,'MA':21,'PI':22,'CE':23,'RN':24,'PB':25,
    'PE':26,'AL':27,'SE':28,'BA':29,'MG':31,'ES':32,
    'RJ':33,'SP':35,'PR':41,'SC':42,'RS':43,'MS':50,
    'MT':51,'GO':52,'DF':53
}

In [6]:
int(df_pop[df_pop.cidade == 'Luziânia']['2020'].values[0])

211508

In [7]:
int(df_pop[(df_pop.cidade == 'Campo Grande')&(df_pop.uf == 'MS')]['2020'].values[0])

906092

In [8]:
df_pop[(df_pop.cidade == 'Amapá')&(df_pop.uf == 'AP')]

,uf,cidade,id,2001,2002,2003,2004,2005,2006,2007,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
186,AP,Amapá,1600105,7179.0,7219.0,7265.0,7296.0,7413.0,7465.0,7492.0,...,8142.0,8213.0,8483.0,8553.0,8622.0,8690.0,8757.0,9029.0,9109.0,9187.0


In [9]:
df_coor[(df_coor.nome == 'Moema')]

,id,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
3107,3142403,Moema,-19.8387,-45.4127,0,31,4847,37,America/Sao_Paulo


In [10]:
cidade = input('Digite a cidade: ')
estado = input('Digite a UF: ').upper()

#cidade = 'Luziânia'
#estado = 'GO'

latitude = df_coor[(df_coor.nome == cidade)&(df_coor.codigo_uf == dicionario[estado])].latitude.values[0]
latitude_text = latitude

longitude = df_coor[(df_coor.nome == cidade)&(df_coor.codigo_uf == dicionario[estado])].longitude.values[0]
longitude_text = longitude

#longitude = longitude + .020
pop = int(df_pop[(df_pop.cidade == cidade)&(df_pop.uf == estado)]['2020'].values[0])
#pop = int(df_pop[df_pop.cidade == cidade]['2020'].values[0])
#pop2 = df_pop[df_pop.cidade == cidade]['2020'].values[0]
print(pop)

start = None
start_mini = None

if pop < 50000:
    start = 14
    # longitude = longitude + .0200
    # latitude_text = latitude + .0200
    # longitude_text = longitude + .0100
    start_mini = -10
elif 50000 < pop < 100000:
    start = 13
    # latitude = latitude + .0100
    # longitude = longitude + .0400
    # latitude_text = latitude + .0400
    # longitude_text = longitude + .0250
    start_mini = -9
elif 100000 < pop < 250000:
    start = 13
    # latitude = latitude + .0400
    # longitude = longitude + .1000
    # latitude_text = latitude + .0800
    # longitude_text = longitude + .0400
    start_mini = -8
elif 250000 < pop < 500000:
    start = 12
    # latitude = latitude + .0400
    # longitude = longitude + .1000
    # latitude_text = latitude + .0800
    # longitude_text = longitude + .0400
    start_mini = -8
elif 500000 < pop < 5000000:
    start = 12
    # latitude = latitude + .0200
    # longitude = longitude + .1000
    # latitude_text = latitude + .0800
    # longitude_text = longitude + .0400
    start_mini = -8 
elif 5000000 < pop < 10000000:
    start = 13
    # latitude = latitude + .0200
    # longitude = longitude + .0300
    # latitude_text = latitude + .0400
    # longitude_text = longitude + .0250
    start_mini = -7 
elif pop > 10000000:
    start = 11
    # latitude = latitude + .0200
    # longitude = longitude + .1100
    # latitude_text = latitude + .1600
    # longitude_text = longitude + .1000
    start_mini = -7 

print(start)
#print(latitude, longitude)

#
mapa = folium.Map(
    #location=[-16.3592,-46.9022],
    location=[latitude,longitude],
    zoom_start=start,
    width=800,
    height=600,
    left='0%',
    #tiles='mapquestopen'
)

minimap = plugins.MiniMap(
    zoom_level_offset=start_mini,
)

# back = folium.Rectangle(
#     bounds=[
#         [latitude-.100,longitude+.08],
#         [latitude+.080,longitude+.250],
#         [latitude+.080,longitude+.250],
#         [latitude-.100,longitude+.08]
#     ],
#     color='#ff7800',
#     fill=True,
#     fill_color='#ffff00',
#     fill_opacity=0.2,
# ).add_to(mapa)




# folium.map.Marker(
#     [latitude_text, longitude_text],
#     #tooltip='center',
#     #popup=folium.Popup(f'<div style="font-size: 10pt">{cidade}</div>'),
#     icon=DivIcon(
#         icon_size=(250,36),
#         icon_anchor=(0,0),
#         html=f'''<div style="font-size: 20pt">{cidade}</div>
#         <div style="font-size: 14pt">Ano: {2020}</div>
#         <div style="font-size: 12pt">População: {pop} habitantes</div>
#         <div style="font-size: 12pt">Natalidade: {pop} nascimentos</div>
#         <div style="font-size: 12pt">Mortalidade: {pop} óbitos</div>''',
#         #html=f'<div style="font-size: 10pt">{pop}</div>',
#         )
#     ).add_to(mapa)

mapa.add_child(minimap)

Digite a cidade:  Itumbiara
Digite a UF:  go


105809
13


In [11]:
dicionario_go = {}
for i in df_pop[df_pop.uf == 'GO'].index:
    perc = f'{((df_pop.loc[i][-1]/df_pop.loc[i][3])-1)*100:.2f}'
    dicionario_go[df_pop.loc[i][1]] = [df_pop.loc[i][1], df_pop.loc[i][2], float(perc)]

#dicionario_go

In [12]:
df_go = pd.DataFrame(dicionario_go.values(),columns=['cidade', 'id', 'percentual'])
df_go

,cidade,id,percentual
0,Abadia de Goiás,5200050,71.28
1,Abadiânia,5200100,75.39
2,Acreúna,5200134,19.98
3,Adelândia,5200159,0.60
4,Água Fria de Goiás,5200175,27.97
...,...,...,...
241,Varjão,5221908,9.56
242,Vianópolis,5222005,20.85
243,Vicentinópolis,5222054,45.39
244,Vila Boa,5222203,88.93


In [13]:
# Pegando o geojson
url_geojson = 'json_geographic_brazil/centro-oeste/go.json'

geo = gpd.read_file(url_geojson)

#geo

In [14]:
# Levando os dados da taxa de mortalidade para dentro do objeto geoespacial
geo_go = pd.merge(
    left = geo,
    right = df_go.filter(items=['id','percentual']), 
    on='id'
)

#geo_go

In [19]:
geo_go

,id,name,description,geometry,percentual
0,5200050,Abadia de Goiás,Abadia de Goiás,"POLYGON ((-49.41162 -16.74047, -49.40080 -16.7...",71.28
1,5200100,Abadiânia,Abadiânia,"POLYGON ((-48.84137 -16.00442, -48.79594 -16.0...",75.39
2,5200134,Acreúna,Acreúna,"POLYGON ((-50.35187 -17.30217, -50.34408 -17.3...",19.98
3,5200159,Adelândia,Adelândia,"POLYGON ((-50.13541 -16.34422, -50.13196 -16.3...",0.60
4,5200175,Água Fria de Goiás,Água Fria de Goiás,"POLYGON ((-47.75897 -14.63961, -47.75319 -14.6...",27.97
...,...,...,...,...,...
241,5221908,Varjão,Varjão,"POLYGON ((-49.68360 -16.95085, -49.67304 -16.9...",9.56
242,5222005,Vianópolis,Vianópolis,"POLYGON ((-48.43125 -16.62755, -48.42527 -16.6...",20.85
243,5222054,Vicentinópolis,Vicentinópolis,"POLYGON ((-49.85005 -17.57682, -49.84311 -17.5...",45.39
244,5222203,Vila Boa,Vila Boa,"POLYGON ((-47.11019 -14.67150, -47.11607 -14.6...",88.93


In [15]:
# cidade = 'Porto Seguro'
# for i in geo_go.index:
#     city = geo_go.loc[i].name
#     if city == cidade:
#         cidade_cord = geo_go.loc[i].geometry
        
#cidade_cord

In [16]:
# # encontrando as coordenadas da cidade escolhida
# for i in range(len(geo_go['features'])):
#     city = geo_go['features'][i]['properties']['name']
#     #print(city)
#     if city == cidade:
#         cidade_cord = (geo_go['features'][i]['geometry']['coordinates'])

In [24]:
# Escolhendo a cidade e estado
cidade = input('Digite a cidade: ')
estado = input('Digite a UF: ').upper()


# encontrando as coordenadas da cidade escolhida
for i in geo_go.index:
    city = geo_go.loc[i].name
    if city == cidade:
        cidade_cord = geo_go.loc[i].geometry




latitude = df_coor[(df_coor.nome == cidade)&(df_coor.codigo_uf == dicionario[estado])].latitude.values[0]
latitude_text = latitude

longitude = df_coor[(df_coor.nome == cidade)&(df_coor.codigo_uf == dicionario[estado])].longitude.values[0]
longitude_text = longitude

#longitude = longitude + .020
pop = int(df_pop[(df_pop.cidade == cidade)&(df_pop.uf == estado)]['2020'].values[0])


start = None
start_mini = None

if pop < 50000:
    start = 14
    start_mini = -10
elif 50000 < pop < 100000:
    start = 13
    start_mini = -9
elif 100000 < pop < 250000:
    start = 13
    start_mini = -8
elif 250000 < pop < 500000:
    start = 12
    start_mini = -8
elif 500000 < pop < 5000000:
    start = 12
    start_mini = -8 
elif 5000000 < pop < 10000000:
    start = 13
    start_mini = -7 
elif pop > 10000000:
    start = 11
    start_mini = -7 

print(start)
#print(latitude, longitude)

#
mapa = folium.Map(
    location=[latitude,longitude],
    zoom_start=start,
    width=800,
    height=600,
    left='0%',
)

# Criando um minimap
minimap = plugins.MiniMap(
    zoom_level_offset=start_mini,
)

# Criando a camada Choroplet
choro = folium.Choropleth(
    geo_data = geo_go[geo_go.name == cidade],
    name ='choropleth',
    bins=100,
    data = df_go[df_go.cidade == cidade],
    columns = ['id','percentual'],
    key_on = 'feature.properties.id',
    fill_color = 'YlOrBr',
    nan_fill_color = 'white',
    #nan_fill_opacity = 0.05,
    fill_opacity = 0.02,
    line_opacity = 0.9,
    line_color = 'black', 
    line_weight = 0.25,
    #legend_name = 'Variação percentual da população entre 2001 e 2020',
    highlight = True,
    reset = True,

).add_to(mapa)

#Inserindo os rotulos ao passar o mouse
choro.geojson.add_child(
  folium.features.GeoJsonTooltip(
      fields = ['name','id','percentual'],
      aliases = ['Município:','id:', 'Percentual:'],
      labels = True,
      localize = True,
      sticky = True,
      style=('background-color: black; color: white; font-family: Arial')
      
  )
)

mapa.add_child(minimap)

Digite a cidade:  Senador Canedo
Digite a UF:  go


13


In [18]:
# m = folium.Map(
#     #location=[-16.3592,-46.9022],
#     location=[-16.3592,-46.8622],
#     zoom_start=13,
#     width=1000,
#     height=600,
#     left='0%',
#     #tiles='Cartodbpositron'
# )
# mg = folium.Map(
#     location=[-18.1,-44.38],
#     zoom_start=13,
#     width=1000,
#     height=600,
#     left='0%',
#     #tiles='Cartodbpositron'
# )

# minimap = plugins.MiniMap(
#     zoom_level_offset=-9,
#     #collapsed_width=0,
#     #tile_layer="cartodbdark_matter",
#     #toggle_display=True
# )
# m.add_child(minimap)


# folium.Circle(
# radius=5,
# location=[-16.356418112875975, -46.904344774398986],
# color='crimson',
# fill=False,).add_to(m)
# m

# folium.CircleMarker(
#     location=(-16.356418112875975, -46.904344774398986),
#     radius=80, 
#     fill_color='red').add_to(m)
m

NameError: name 'm' is not defined